In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def solve_euler_explicit(f,t0,tf,x0,dt):#t0 et tf temps de début et de fin de la simulation
    X=x0#initialisation
    t=t0
    x_arr=[X]
    t_arr=[t]#espace de stockage
    while t<tf:
        X=X+dt*f(X,t)
        t=t+dt
        x_arr.append(X)
        t_arr.append(t)
        return np.array(t_arr), np.array(x_arr)
    
    
